<a href="https://colab.research.google.com/github/Mikklo/jpg2pdf_colab/blob/main/OCR_Gdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the virtual machine
Only run this when opening this notebook again in a browser. It will start a virtual machine and install everything needed. This make take a couple of minutes. Wait  for the checkmark to the left of the script.

In [ ]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!apt install tesseract-ocr-swe
!apt install ghostscript
!pip install pytesseract
!pip install PyMuPDF
!pip install PyPDF2
!pip install ocrmypdf

The following two scripts are also only needed if a new machine has just been created
---



In [ ]:
from PIL import Image
import os
import pandas as pd
import shutil
from PyPDF2 import PdfMerger
from google.colab import drive
%load_ext google.colab.data_table

This will require access to your google drive, it will create two folders, output_pdf and output_ocr and use them for storing the output

In [ ]:
drive.mount('/content/gdrive')
input_directory = './input_jpg'
output_directory = '/content/gdrive/MyDrive/output_pdf'
output_ocr_directory = '/content/gdrive/MyDrive/output_ocr'
os.makedirs(input_directory, exist_ok=True)
os.makedirs(output_directory, exist_ok=True)
os.makedirs(output_ocr_directory, exist_ok=True)



---
# Converting the files

**Start here if you already have the machine running**
   
   
---


This will convert all the files located in **input_jpg** to pdf.

Use the menu to the left to upload files to that folder, make sure all files are fully uploaded before proceeding.

In [ ]:
for filename in os.listdir(input_directory):
    if filename.endswith(".jpg"):
        img_path = os.path.join(input_directory, filename)
        pdf_path = os.path.join(output_directory, filename.replace(".jpg", ".pdf"))

        img = Image.open(img_path)
        pdf = img.convert("RGB")
        pdf.save(pdf_path, "PDF", resolution=100.0)

print("Batch conversion complete.")

Verify the correct content of **output_pdf** for OCR scanning.

In [ ]:
file_list = []
for filename in os.listdir(output_directory):
    if filename.endswith("pdf"):
      file_list.append(filename)

print("The following files will be converted : ")
pd.DataFrame(file_list, columns=["Filename"])

This adds OCR on top of the images in the pdf files and outputs to output_ocr. Make sure to wait for the checkmark before downloading any pdf files.



> **THIS WILL TAKE TIME**



In [ ]:
for filename in file_list:
  print("Converting : " + filename)
  input_filename = output_directory+'/'+filename
  output_filename = output_ocr_directory+'/'+filename
  !ocrmypdf --force-ocr -l swe "{input_filename}" "{output_filename}"

Combine the OCR scanned PDFs into one single file named combined_ocr.pdf in the **output_ocr** folder

In [ ]:
combined_pdf_path = output_ocr_directory+'/'+'combined_ocr.pdf'
merger = PdfMerger()

for filename in os.listdir(output_ocr_directory):
    if filename.endswith(".pdf"):
        pdf_path = os.path.join(output_ocr_directory, filename)
        merger.append(pdf_path)

merger.write(combined_pdf_path)
merger.close()

print("PDF files combined into a single PDF.")





---





🛑 To reset and clear out all directories and upload new images run this script 🛑

In [ ]:
def empty_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error emptying {file_path}: {e}")

empty_directory(input_directory)
empty_directory(output_directory)
empty_directory(output_ocr_directory)
print("All files in input, output, and OCR output directories have been emptied.")
